In [1]:
texto = """Fundada em 1 de abril de 1976 em Cupertino, Califórnia, e incorporada 3 de janeiro de 1977, a empresa foi anteriormente 
denominada Apple Computer nos seus primeiros 30 anos, mas retirou a palavra "Computer" em 9 de janeiro de 2007, para refletir a 
contínua expansão da empresa no mercado de eletrônicos de consumo, além de seu foco tradicional em computadores pessoais. Em 
setembro de 2010, a Apple tinha 46,6 mil empregados em tempo integral e 2 800 temporários empregados em tempo integral em todo o 
mundo e tinha vendas anuais mundiais de 65,23 bilhões de dólares."""

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from langchain_core.utils.function_calling import convert_to_openai_function

class Acontecimento(BaseModel):
    """Identifica os acontecimentos que ocorreram no texto"""
    data: str = Field(description="Data do acontecimento no formato YYYY-MM-DD")
    acontecimento: str = Field(description="Acontecimento extraido do texto")

class ListaAcontecimentos(BaseModel):
    """Identifica os acontecimentos que ocorreram no texto"""
    acontecimentos: List[Acontecimento] = Field(description="Lista de acontecimentos presentes no texto")

tool_acontecimentos = convert_to_openai_function(ListaAcontecimentos)
tool_acontecimentos

c:\Users\johns\OneDrive\Documentos\Projetos\GitHub\Regdoor_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


{'name': 'ListaAcontecimentos',
 'description': 'Identifica os acontecimentos que ocorreram no texto',
 'parameters': {'type': 'object',
  'properties': {'acontecimentos': {'description': 'Lista de acontecimentos presentes no texto',
    'type': 'array',
    'items': {'description': 'Identifica os acontecimentos que ocorreram no texto',
     'type': 'object',
     'properties': {'data': {'description': 'Data do acontecimento no formato YYYY-MM-DD',
       'type': 'string'},
      'acontecimento': {'description': 'Acontecimento extraido do texto',
       'type': 'string'}},
     'required': ['data', 'acontecimento']}}},
  'required': ['acontecimentos']}}

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia os aontecimentos contidos no texto, elas devem ser extraidas integralmente"),
    ("user", "{input}")
])
chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_acontecimentos], function_call={"name": "ListaAcontecimentos"}))
chain.invoke({"input": texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"acontecimentos":[{"data":"1976-04-01","acontecimento":"Fundação da empresa em Cupertino, Califórnia."},{"data":"1977-01-03","acontecimento":"Incorporação da empresa."},{"data":"2007-01-09","acontecimento":"Retirada da palavra \'Computer\' do nome da empresa."},{"data":"2010-09","acontecimento":"Apple tinha 46,6 mil empregados em tempo integral e 2 800 temporários empregados em tempo integral em todo o mundo, com vendas anuais mundiais de 65,23 bilhões de dólares."}]}', 'name': 'ListaAcontecimentos'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 319, 'total_tokens': 461, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'l

#### Incorporando OutputParser

In [4]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt | 
    chat.bind(functions=[tool_acontecimentos], function_call={"name": "ListaAcontecimentos"}) | 
    JsonOutputFunctionsParser())

chain.invoke({"input": texto})

{'acontecimentos': [{'data': '1976-04-01',
   'acontecimento': 'Fundação da empresa em Cupertino, Califórnia'},
  {'data': '1977-01-03', 'acontecimento': 'Incorporação da empresa'},
  {'data': '2007-01-09',
   'acontecimento': 'Remoção da palavra "Computer" do nome da empresa para refletir a expansão no mercado de eletrônicos de consumo'},
  {'data': '2010-09',
   'acontecimento': 'A Apple tinha 46,6 mil empregados em tempo integral e 2.800 temporários em todo o mundo, com vendas anuais de 65,23 bilhões de dólares'}]}

#### Incorporando OutputParser com mais formatação

In [5]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt | 
    chat.bind(functions=[tool_acontecimentos], function_call={"name": "ListaAcontecimentos"}) | 
    JsonKeyOutputFunctionsParser(key_name="acontecimentos"))

chain.invoke({"input": texto})

[{'data': '1976-04-01',
  'acontecimento': 'Fundação da empresa em Cupertino, Califórnia.'},
 {'data': '1977-01-03', 'acontecimento': 'Incorporação da empresa.'},
 {'data': '2007-01-09',
  'acontecimento': "Retirada da palavra 'Computer' do nome da empresa."},
 {'data': '2010-09',
  'acontecimento': 'Apple tinha 46,6 mil empregados em tempo integral e 2 800 temporários empregados em tempo integral em todo o mundo, com vendas anuais mundiais de 65,23 bilhões de dólares.'}]

## Extraindo informações de uma pagina web

In [6]:
from langchain_community.document_loaders.web_base import WebBaseLoader
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function
from dataclasses import field

loader = WebBaseLoader("https://hub.asimov.academy/blog/")
page = loader.load()

class BlogPost(BaseModel):
    """Informações sobre um post de blog"""
    titulo: str = Field(description="Título do post")
    autor: str = Field(description="Autor do post")
    data_publicacao: str = Field(description="Data de publicação do post")

class BlogSite(BaseModel):
    """Lista de blog posts de um site"""
    posts: List[BlogPost] = Field(description="Lista de posts do blog")

tool_blog = convert_to_openai_function(BlogSite)
tool_blog

USER_AGENT environment variable not set, consider setting it to identify your requests.


{'name': 'BlogSite',
 'description': 'Lista de blog posts de um site',
 'parameters': {'type': 'object',
  'properties': {'posts': {'description': 'Lista de posts do blog',
    'type': 'array',
    'items': {'description': 'Informações sobre um post de blog',
     'type': 'object',
     'properties': {'titulo': {'description': 'Título do post',
       'type': 'string'},
      'autor': {'description': 'Autor do post', 'type': 'string'},
      'data_publicacao': {'description': 'Data de publicação do post',
       'type': 'string'}},
     'required': ['titulo', 'autor', 'data_publicacao']}}},
  'required': ['posts']}}

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia da pagina todos os posts do blog com autor e data da publicação"),
    ("user", "{input}")
])

chain = (
    prompt | 
    chat.bind(functions=[tool_blog], function_call={"name": "BlogSite"}) | 
    JsonKeyOutputFunctionsParser(key_name="posts"))

chain.invoke({"input": page})

[{'titulo': 'Código binário: o que é e como manipular essa linguagem ?',
  'autor': 'Rebeca Honório',
  'data_publicacao': '5 horas atrás'},
 {'titulo': 'O que é a Hugging Face e sua importância na inteligência artificial',
  'autor': 'Rebeca Honório',
  'data_publicacao': '4 dias atrás'},
 {'titulo': 'Tuplas em Python: o guia completo para iniciantes',
  'autor': 'Rebeca Honório',
  'data_publicacao': '7 dias atrás'},
 {'titulo': 'Open Source: o impacto transformador na tecnologia e o papel do Python',
  'autor': 'Rebeca Honório',
  'data_publicacao': '1 dia atrás'},
 {'titulo': 'DeepSeek: conheça a IA que está revolucionando o mercado',
  'autor': 'Rebeca Honório',
  'data_publicacao': '5 dias atrás'},
 {'titulo': 'Big Data: o que é, como funciona e sua importância no mundo digital',
  'autor': 'Rebeca Honório',
  'data_publicacao': '5 dias atrás'},
 {'titulo': 'Plotly: a biblioteca para visualização de dados em Python',
  'autor': 'Rebeca Honório',
  'data_publicacao': '5 dias atrás